In [2]:
import sys
r_path_data = "../new_codebase/src/models/kmeans/"
sys.path.append(r_path_data)
from kmeans import *

In [3]:
import time

In [4]:
% matplotlib inline

In [50]:
result=get_cluster_results('ovasarhelyi','pre-summer', 'all', features, nc=4)

    label  ratio
3  360508  52.59
4  135325  19.74
0  127156  18.55
1   58465   8.53
2    4059   0.59


In [5]:
season='pre-summer'
country= 'all'
nc=4
#create_basic_description(result, season, country)

In [6]:
def calculate_cluster_size(kmeans_res, var):
    cluster_results=pd.DataFrame(kmeans_res[var].value_counts())
    ratio=np.round(cluster_results/cluster_results.sum()*100, 2).rename(columns={var:"ratio"})
    return cluster_results.join(ratio)

In [7]:
def cluster_size(result, var):
    a=calculate_cluster_size(result, var)
    a['cus']=a.index
    return a

In [8]:
def create_basic_description(result, season, country, var):
    if country!='all':
        return f'In the {season} season {(len(result))} tourists visited Tuscany from {country}, we managed to cluster {np.round(len(result[pd.notnull(result[var])])/len(result)*100,2)} % of them.'
    else:
        return f'In the {season} season {(len(result))} tourists visited Tuscany, we managed to cluster {np.round(len(result[pd.notnull(result[var])])/len(result)*100,2)} % of them.'

In [9]:
def get_top_nationalities(result, n):
    nat_freq=pd.DataFrame(result['country'].value_counts())
    ratios=nat_freq[:n]/nat_freq.sum()*100
    res='The most common visitors are from'
    for i in range(0,len(ratios)):
        if i!=len(ratios)-1:
            res=res+f' {ratios.index[i]} ({np.round(ratios.country[i],2)}%),'
        else:
            res=res+f' and {ratios.index[i]} ({np.round(ratios.country[i],2)}%).'
    return res

In [10]:
get_top_nationalities(result, 4)

'The most common visitors are from United States (13.37%), Germany (12.55%), United Kingdom (9.05%), and France (8.87%).'

In [10]:
def get_cluster_baiscs(result, var):
    clusters=calculate_cluster_size(result, var)
    res=f'We have {len(clusters)} clusters,'
    for i in zip(range(0,len(clusters)), ['first', 'second', "third", 'forth', 'fifth'][:len(clusters)]):
        if i[0]!=len(clusters)-1:
            #print(i)
            res=res+f' the {i[1]} cluster represents {clusters.ratio[i[0]]}%,'
        else:
            res=res+f' and the {i[1]} cluster is {clusters.ratio[i[0]]}%.'
    return res

In [11]:
def get_cluster_country_distr(result, var):
    return pd.crosstab(result.country, result[var]).apply(lambda x: x / x.sum()*100, 0).round(2)

In [12]:
def hours_tusc(result, var):
    return np.round(pd.DataFrame(result.groupby(var)[['hrs_in_tusc']].mean())/24)

In [13]:
def get_places_at_least4_hours(result, cluster, var):
    hours=get_hours_by_cities(result, var)
    four_hours_top=hours.sort_values(cluster, ascending=False)
    cities=four_hours_top[four_hours_top[cluster]>4].index
    if len(cities)==0:
        res='but nowhere more than 4 hours. '
    else:
        res='and out of it at least a half day in'
        i=0
        for city in cities:
            i+=1
            if city=='coast':
                city='the coast'
            if i!=len(cities) and len(cities)!=1:
                res=res+f' {city.title()},'
            elif i==len(cities) and i!=1:
                res=res+f' and {city.title()}. '
            else:
                res=res+f' {city.title()}. '
    return res

In [14]:
def get_hours_by_cities(result, var):
    return pd.DataFrame(result.groupby(var)[['arezzo', 'florence', 'grosseto', 'livorno',
       'lucca', 'pisa', 'pistoia', 'siena', 'coast']].mean()/60).T

In [15]:
def cluster_mcc_ratio(result, nc, n, var):
    rel1=get_cluster_country_distr(result, var)
    hours=hours_tusc(result, var)
    res=""
    for i in zip(range(0,nc), [" first", ' second', ' third', ' forth', ' fifth'][:nc]):
        res=res+f"By the number of uniqiue visitors the{i[1]} cluster's top {n} countries are; "
        rel=rel1.sort_values(i[0],ascending=False)[:n]
        for j in range(0,n):
            if j!=n-1:
                res=res+f'{rel[i[0]].index[j]} ({rel[i[0]][j]}%), '
            else:
                res=res+f'and {rel[i[0]].index[j]} ({rel[i[0]][j]}%). '
                res=res+f'This cluster spends in average {int(hours.hrs_in_tusc[i[0]])} days in Tuscany, '
                res=res+get_places_at_least4_hours(result, i[0], var)
    return res

In [17]:
season, country

('pre-summer', 'all')

In [18]:
#get_cluster_country_distr(result, "label")

In [19]:
var="label"
print(create_basic_description(result, season, country, var))
print(get_top_nationalities(result, 6))
print(get_cluster_baiscs(result, var))
print(cluster_mcc_ratio(result,4, 5, var))

In the pre-summer season 725563 tourists visited Tuscany, we managed to cluster 94.49 % of them.
The most common visitors are from United States (13.37%), Germany (12.55%), United Kingdom (9.05%), France (8.87%), Netherlands (6.78%), and China (5.14%).
We have 5 clusters, the first cluster represents 59.54%, the second cluster represents 13.27%, the third cluster represents 0.6%, the forth cluster represents 9.39%, and the fifth cluster is 17.2%.
By the number of uniqiue visitors the first cluster's top 5 countries are; Germany (13.16%), United States (11.79%), United Kingdom (10.42%), France (9.67%), and Netherlands (6.66%). This cluster spends in average 4 days in Tuscany, and out of it at least a half day in Florence, and The Coast. By the number of uniqiue visitors the second cluster's top 5 countries are; Germany (18.38%), United States (15.34%), France (9.76%), Switzerland (8.15%), and United Kingdom (6.11%). This cluster spends in average 8 days in Tuscany, and out of it at leas

In [20]:
def get_kmeans_description(result, season, country, var):
    print(create_basic_description(result, season, country, var)+' '+
    get_top_nationalities(result, 6) +" "+get_cluster_baiscs(result, var)+ ' \n'+
    cluster_mcc_ratio(result,4, 5, var))

In [21]:
get_kmeans_description(result,season, country, "label")

In the pre-summer season 725563 tourists visited Tuscany, we managed to cluster 94.49 % of them. The most common visitors are from United States (13.37%), Germany (12.55%), United Kingdom (9.05%), France (8.87%), Netherlands (6.78%), and China (5.14%). We have 5 clusters, the first cluster represents 59.54%, the second cluster represents 13.27%, the third cluster represents 0.6%, the forth cluster represents 9.39%, and the fifth cluster is 17.2%. 
By the number of uniqiue visitors the first cluster's top 5 countries are; Germany (13.16%), United States (11.79%), United Kingdom (10.42%), France (9.67%), and Netherlands (6.66%). This cluster spends in average 4 days in Tuscany, and out of it at least a half day in Florence, and The Coast. By the number of uniqiue visitors the second cluster's top 5 countries are; Germany (18.38%), United States (15.34%), France (9.76%), Switzerland (8.15%), and United Kingdom (6.11%). This cluster spends in average 8 days in Tuscany, and out of it at lea

# Trajectory

In [22]:
path_to_result='/mnt/data/ovasarhelyi/TPT_tourism/new_codebase/src/models/sequence_analysis/data/clustering_results/'

In [23]:
d=pd.read_csv(path_to_result+'cluster_results_hungary_winter.csv')

In [32]:
season, country

('pre-summer', 'all')

In [24]:
import sys
r_path_data = "../new_codebase/src/utils/load_data/"
sys.path.append(r_path_data)
from load_dataframes import *

In [25]:
def trajectory_cluster_description(result, var):
    hours=hours_tusc(result, var)
    nc=len(hours)
    res=''
    for i in zip(range(1,nc), [" first", ' second', ' third', ' forth', ' fifth'][:nc]):
        res=res + f'The {i[1]} cluster '
        res=res + f'spends in average {int(hours.hrs_in_tusc[(i[0])])} days in Tuscany, '
        res=res+get_places_at_least4_hours(result, i[0], var)
    return res 

In [26]:
def join_customer_features(traj_result, username,season, country):
    user_features=get_k_means_data(username,season, country).set_index("customer_nr")
    features_with_trajectory=user_features.join(traj_result.set_index('customer_nr')[["cluster"]])
    return features_with_trajectory

In [27]:
def get_trajectory_description(traj_result, username, season, country, var):
    result=join_customer_features(traj_result, username, season, country)
    print(create_basic_description(result, season, country, var)+" "+
    trajectory_cluster_description(result, var))

In [28]:
#traj_result

In [34]:
traj_result=d
username='ovasarhelyi'
country = 'hungary'
season = 'winter'
var = 'cluster'
get_trajectory_description(traj_result, username, season, country, var)

In the winter season 7175 tourists visited Tuscany from hungary, we managed to cluster 83.62 % of them. The  first cluster spends in average 8 days in Tuscany, and out of it at least a half day in Florence, and Pisa. The  second cluster spends in average 7 days in Tuscany, but nowhere more than 4 hours. The  third cluster spends in average 2 days in Tuscany, and out of it at least a half day in Florence. The  forth cluster spends in average 1 days in Tuscany, and out of it at least a half day in Pisa, and Florence. 


# Airport  arrival

In [16]:
result=get_cluster_results('ovasarhelyi','summer', 'france', features, nc=4)

    label  ratio
2  148087  49.75
1   93249  31.33
0   31087  10.44
3   23557   7.91
4    1661   0.56


In [17]:
import geopandas as gpd
from shapely.geometry import Point

def create_airports(data):
    """
    Exmaple: 
    data = airport_cities_d = {"airport": ['Pisa', 'Florence'], 
         "lat": [43.7228, 43.7696], "lon": [10.4017, 11.2558]}
    """
    
    airport_cities = pd.DataFrame(data)
    geometry = [Point(xy) for xy in zip(airport_cities.lon, airport_cities.lat)]
    airport_cities = airport_cities.drop(['lon', 'lat'], axis=1)
    crs = {'init': 'epsg:4326'}
    geo_airport_cities = gpd.GeoDataFrame(airport_cities, crs=crs, geometry=geometry)
    return geo_airport_cities
    

In [19]:
airport_cities_d = {"airport": ['Pisa', 'Florence'], 
         "lat": [43.7228, 43.7696], "lon": [10.4017, 11.2558]}

In [20]:
geo_airport_cities=create_airports(airport_cities_d)

In [21]:
type(geo_airport_cities)

geopandas.geodataframe.GeoDataFrame

In [22]:
crs={'init': 'epsg:4326'}

In [23]:
len(result)

305080

In [26]:
geometry_st = [Point(xy) for xy in zip(result.start_lon, result.start_lat)]
geometry_end = [Point(xy) for xy in zip(result.end_lon, result.end_lat)]

In [29]:
len(geometry_end)

305080

In [30]:
geo_st = gpd.GeoDataFrame(geometry_st, crs=crs, geometry=geometry_st)[['geometry']]

In [32]:
geo_st = gpd.GeoDataFrame(geometry_st, crs=crs, geometry=geometry_st)[['geometry']]
geo_end = gpd.GeoDataFrame(geometry_end, crs=crs, geometry=geometry_end)[['geometry']]

In [33]:
#result

In [37]:
st_pisa = pd.DataFrame(geo_st.within(geo_airport_cities['geometry'].loc[0].buffer(0.1)))
st_pisa.index=result.index
result['geometry_st_pisa'] = st_pisa
end_pisa = pd.DataFrame(geo_end.within(geo_airport_cities['geometry'].loc[0].buffer(0.1)))
end_pisa.index=result.index
result['geometry_end_pisa'] = end_pisa
result.head()

,mcc,hr_arvl_tusc,day_of_wk_arvl_tusc,mon_arvl_tusc,day_arvl_tusc,loc_arvl_tusc,hr_arvl_italy,day_of_wk_arvl_italy,mon_arvl_italy,day_arvl_italy,...,lucca,pisa,pistoia,siena,coast,num_attrs,country,label,geometry_st_pisa,geometry_end_pisa
customer_nr,,,,,,,,,,,,,,,,,,,,,
858778.0,208.0,10.0,1.0,6.0,19.0,3041313.0,10.0,1.0,6.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,France,2.0,False,False
1001946.0,208.0,14.0,1.0,6.0,19.0,3036981.0,14.0,1.0,6.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,France,2.0,False,False
1150164.0,208.0,17.0,5.0,6.0,23.0,3600583.0,17.0,5.0,6.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,France,2.0,False,False
1481050.0,208.0,10.0,2.0,6.0,20.0,3627917.0,10.0,2.0,6.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,France,NaN,False,False
2353070.0,208.0,19.0,3.0,8.0,16.0,59186011.0,19.0,3.0,8.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,France,2.0,False,False


In [43]:
st_florence = pd.DataFrame(geo_st.within(geo_airport_cities['geometry'].loc[1].buffer(0.1)))
st_florence.index=result.index
result['geometry_st_fl'] = st_florence
end_florence = pd.DataFrame(geo_end.within(geo_airport_cities['geometry'].loc[1].buffer(0.1)))
end_florence.index=result.index
result['geometry_end_fl'] = end_florence
result.head()

,mcc,hr_arvl_tusc,day_of_wk_arvl_tusc,mon_arvl_tusc,day_arvl_tusc,loc_arvl_tusc,hr_arvl_italy,day_of_wk_arvl_italy,mon_arvl_italy,day_arvl_italy,...,coast,num_attrs,country,label,geometry_st_pisa,geometry_end_pisa,geometry_st_florence,geometry_end_florence,geometry_st_fl,geometry_end_fl
customer_nr,,,,,,,,,,,,,,,,,,,,,
858778.0,208.0,10.0,1.0,6.0,19.0,3041313.0,10.0,1.0,6.0,19.0,...,0.0,0.0,France,2.0,False,False,False,False,False,False
1001946.0,208.0,14.0,1.0,6.0,19.0,3036981.0,14.0,1.0,6.0,19.0,...,0.0,0.0,France,2.0,False,False,False,False,False,False
1150164.0,208.0,17.0,5.0,6.0,23.0,3600583.0,17.0,5.0,6.0,23.0,...,0.0,0.0,France,2.0,False,False,False,False,False,False
1481050.0,208.0,10.0,2.0,6.0,20.0,3627917.0,10.0,2.0,6.0,20.0,...,0.0,0.0,France,NaN,False,False,False,False,False,False
2353070.0,208.0,19.0,3.0,8.0,16.0,59186011.0,19.0,3.0,8.0,16.0,...,0.0,0.0,France,2.0,False,False,False,False,False,False


In [48]:
st_airport = pd.DataFrame(geo_st.within(geo_airport_cities['geometry'].unary_union.buffer(0.1)))
st_airport.index=result.index
result['geometry_st'] = st_airport
end_airport = pd.DataFrame(geo_end.within(geo_airport_cities['geometry'].unary_union.buffer(0.1)))
end_airport.index=result.index
result['geometry_end'] = end_airport

In [49]:
result[["geometry_st_pisa", 'geometry_end_pisa', "geometry_st_fl", "geometry_end_fl"]].sum()

geometry_st_pisa     13706
geometry_end_pisa    13509
geometry_st_fl       40019
geometry_end_fl      37250
dtype: int64

In [44]:
pd.crosstab(result.geometry_st_fl, result.geometry_end_fl).apply(lambda x: x / x.sum()*100, 1).round(2)

geometry_end_fl,False,True
geometry_st_fl,,
False,92.33,7.67
True,57.75,42.25


In [42]:
pd.crosstab(result.geometry_st_pisa, result.geometry_end_pisa).apply(lambda x: x / x.sum()*100, 1).round(2)

geometry_end_pisa,False,True
geometry_st_pisa,,
False,97.17,2.83
True,61.59,38.41


In [ ]:
#r_path_data = "../new_codebase/src/models/kmeans/"
#sys.path.append(r_path_data)
#from kmeans import *

#result = get_cluster_results('ywang99587','summer', 'japan', features, nc=4)
#df_cluster_info = calculate_cluster_size(result)



def near_airport(row):
    airport_cities_d = {"airport": ['Pisa', 'Florence'], 
         "lat": [43.7228, 43.7696], "lon": [10.4017, 11.2558]}
    airport_cities = pd.DataFrame(airport_cities_d)
    geometry = [Point(xy) for xy in zip(airport_cities.lon, airport_cities.lat)]
    airport_cities = airport_cities.drop(['lon', 'lat'], axis=1)
    crs = {'init': 'epsg:4326'}
    geo_airport_cities = GeoDataFrame(airport_cities, crs=crs, geometry=geometry)
    if ((row.within(geo_airport_cities['geometry'].unary_union.buffer(0.1)))):
        return 1
    else:
        return 0
	
	
cluster = result[result['label']==2]
geometry_st = [Point(xy) for xy in zip(cluster.start_lon, cluster.start_lat)]
geometry_end = [Point(xy) for xy in zip(cluster.end_lon, cluster.end_lat)]
cluster_pt_st = cluster.drop(['start_lon', 'start_lat'], axis=1)
cluster_pt_end = cluster.drop(['end_lon', 'end_lat'], axis=1)
crs = {'init': 'epsg:4326'}
geo_st = GeoDataFrame(cluster_pt_st, crs=crs, geometry=geometry_st)
geo_end = GeoDataFrame(cluster_pt_end, crs=crs, geometry=geometry_end)
cluster['start_airport'] = geo_st['geometry'].apply(near_airport,1)
cluster['end_airport'] = geo_end['geometry'].apply(near_airport,1)
cluster.start_airport.sum()/len(cluster)*100
cluster.end_airport.sum()/len(cluster)*100
print(cluster.start_airport.sum()/len(cluster)*100)
print(cluster.end_airport.sum()/len(cluster)*100)
cluster.head()



C